In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os 
import pandas as pd
from lhe_tools import readLHEF,get_event_by_child

In [ ]:
current_folder = os.getcwd()
lhe_file = os.path.join(current_folder, 'unweighted_events.lhe.gz')

childs = readLHEF(lhe_file)



In [ ]:
print(len(childs))

In [ ]:

weight = 1/len(childs)  # Assuming uniform weight for each event
results = []

for child in childs:
    row = {}
    event = get_event_by_child(child)
    # Get all jets (gluon and 5 flavor quarks)
    jets = event.getParticlesByIDs([21, 1, 2, 3, 4, 5, -1 , -2, -3, -4, -5]) 
    higgs = event.getParticlesByIDs([25])  
    
    if len(jets) < 2 or len(higgs) != 1:
        print(f"Skipping event with n_jets={len(jets)} and n_higgs={len(higgs)}")
        print([p.pdgid for p in event.particles])
        break

    # Sort jets by PT (descending order)
    jets_sorted = sorted(jets, key=lambda jet: jet.pt, reverse=True)
    

    
    # Get leading jet information (highest PT)
    leading_jet = jets_sorted[0]
    row['Leading_Jet_PT'] = leading_jet.pt
    row['Leading_Jet_Eta'] = leading_jet.eta
    row['Leading_Jet_Phi'] = leading_jet.phi
    row['Leading_Jet_Mass'] = leading_jet.m
    
    # Get subleading jet information (second highest PT)
    subleading_jet = jets_sorted[1]
    row['Subleading_Jet_PT'] = subleading_jet.pt
    row['Subleading_Jet_Eta'] = subleading_jet.eta
    row['Subleading_Jet_Phi'] = subleading_jet.phi
    row['Subleading_Jet_Mass'] = subleading_jet.m
    
    # Calculate some useful variables using the built-in methods
    
    # Delta R between leading and subleading jets
    row['Delta_R_Jets'] = leading_jet.delta_R(subleading_jet)
    
    # Delta phi between leading and subleading jets
    row['Delta_Phi_Jets'] = leading_jet.delta_phi(subleading_jet)
    
    # Delta eta between the two leading jets (important for VBF)
    row['Delta_Eta_Jets'] = abs(leading_jet.delta_eta(subleading_jet))
    
    # Invariant mass of the two leading jets using TLorentzVector
    tlv1 = leading_jet.get_tlv()
    tlv2 = subleading_jet.get_tlv()
    dijet_tlv = tlv1 + tlv2
    row['Dijet_Mass'] = dijet_tlv.M()
    
    # Get Higgs information
    met = higgs[0] # Assume higgs to invisible decay
    row['MET_PT'] = met.pt
    row['MET_Phi'] = met.phi


    row['event_weight'] = weight
    results.append(row)

results = pd.DataFrame.from_records(results)

In [ ]:
results.head()

In [ ]:
results.shape

In [ ]:
vbf_met_results = results
vbf_met_results.to_csv('vbf_met_preselection.csv', index=False)
qcd_results = pd.read_csv('qcd_preselection.csv')
zjets_results = pd.read_csv('z_jets_preselection.csv')
wjets_results = pd.read_csv('w_jets_preselection.csv')


In [ ]:

# Cross-sections
xs_zjets = 5.89
xs_wjets = 18.74
xs_total = xs_zjets + xs_wjets

# Reweight the datasets according to cross-sections
zjets_results_weighted = zjets_results.copy()
zjets_results_weighted['event_weight'] = zjets_results['event_weight'] * (xs_zjets / xs_total)

wjets_results_weighted = wjets_results.copy()
wjets_results_weighted['event_weight'] = wjets_results['event_weight'] * (xs_wjets / xs_total)

# Concatenate Z+jets and W+jets to create V+jets
vjets_results = pd.concat([zjets_results_weighted, wjets_results_weighted], ignore_index=True)

print(f"Z+jets weight factor: {xs_zjets / xs_total:.3f}")
print(f"W+jets weight factor: {xs_wjets / xs_total:.3f}")
print(f"Z+jets events: {len(zjets_results)}")
print(f"W+jets events: {len(wjets_results)}")
print(f"V+jets events: {len(vjets_results)}")

In [ ]:
# Create 5x2 comparison plots between VBF, QCD, and V+jets
fig, axes = plt.subplots(5, 2, figsize=(15, 25))

# Plot 1: Leading Jet PT comparison
pt_min, pt_max = 20, 800
bin_width = 10
n_bins = int((pt_max - pt_min) / bin_width)
pt_bins = np.linspace(pt_min, pt_max, n_bins + 1)

axes[0, 0].hist(qcd_results['Leading_Jet_PT'], bins=pt_bins, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], range=(pt_min, pt_max), histtype='step', linewidth=2)
axes[0, 0].hist(vjets_results['Leading_Jet_PT'], bins=pt_bins, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], range=(pt_min, pt_max), histtype='step', linewidth=2)
axes[0, 0].hist(results['Leading_Jet_PT'], bins=pt_bins, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], range=(pt_min, pt_max), histtype='step', linewidth=2, linestyle='--')

# Add vertical line for leading jet pT cut
axes[0, 0].axvline(x=80, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Leading Jet Cut (80 GeV)')
axes[0, 0].set_xlabel(r'$p_{T}(j_{1})$ [GeV]', loc = 'right')
axes[0, 0].set_ylabel('Frecuency [A.U]')
axes[0, 0].set_xlim(pt_min, pt_max)
axes[0, 0].set_yscale('log')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Save individual plot for Leading Jet PT
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Leading_Jet_PT'], bins=pt_bins, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], range=(pt_min, pt_max), histtype='step', linewidth=2)
plt.hist(vjets_results['Leading_Jet_PT'], bins=pt_bins, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], range=(pt_min, pt_max), histtype='step', linewidth=2)
plt.hist(results['Leading_Jet_PT'], bins=pt_bins, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], range=(pt_min, pt_max), histtype='step', linewidth=2, linestyle='--')
plt.axvline(x=80, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Leading Jet Cut (80 GeV)')
plt.xlabel(r'$p_{T}(j_{1})$ [GeV]', loc = 'right')
plt.ylabel('Frecuency [A.U]')
plt.xlim(pt_min, pt_max)
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/leading_jet_pt_comparison.pdf')
plt.close()

# Plot 2: Subleading Jet PT comparison
axes[0, 1].hist(qcd_results['Subleading_Jet_PT'], bins=pt_bins, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], range=(pt_min, pt_max/2), histtype='step', linewidth=2)
axes[0, 1].hist(vjets_results['Subleading_Jet_PT'], bins=pt_bins, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], range=(pt_min, pt_max/2), histtype='step', linewidth=2)
axes[0, 1].hist(results['Subleading_Jet_PT'], bins=pt_bins, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], range=(pt_min, pt_max/2), histtype='step', linewidth=2, linestyle='--')
# Add vertical line for subleading jet pT cut
axes[0, 1].axvline(x=50, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Subleading Jet Cut (50 GeV)')
axes[0, 1].set_xlabel(r'$p_{T}(j_{2})$ [GeV]', loc='right')
axes[0, 1].set_ylabel('Frecuency [A.U]')
axes[0, 1].set_xlim(pt_min, pt_max/2)
axes[0, 1].set_yscale('log')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Save individual plot for Subleading Jet PT
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Subleading_Jet_PT'], bins=pt_bins, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], range=(pt_min, pt_max/2), histtype='step', linewidth=2)
plt.hist(vjets_results['Subleading_Jet_PT'], bins=pt_bins, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], range=(pt_min, pt_max/2), histtype='step', linewidth=2)
plt.hist(results['Subleading_Jet_PT'], bins=pt_bins, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], range=(pt_min, pt_max/2), histtype='step', linewidth=2, linestyle='--')
plt.axvline(x=50, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Subleading Jet Cut (50 GeV)')
plt.xlabel(r'$p_{T}(j_{2})$ [GeV]', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.xlim(pt_min, pt_max/2)
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/subleading_jet_pt_comparison.pdf')
plt.close()

# Plot 3: Leading Jet Eta comparison
axes[1, 0].hist(qcd_results['Leading_Jet_Eta'], bins=100, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[1, 0].hist(vjets_results['Leading_Jet_Eta'], bins=100, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[1, 0].hist(results['Leading_Jet_Eta'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
axes[1, 0].set_xlabel(r'$\eta(j_{1})$', loc='right')
axes[1, 0].set_ylabel('Frecuency [A.U]')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Save individual plot for Leading Jet Eta
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Leading_Jet_Eta'], bins=100, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_results['Leading_Jet_Eta'], bins=100, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(results['Leading_Jet_Eta'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.xlabel(r'$\eta(j_{1})$', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/leading_jet_eta_comparison.pdf')
plt.close()

# Plot 4: Subleading Jet Eta comparison
axes[1, 1].hist(qcd_results['Subleading_Jet_Eta'], bins=100, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[1, 1].hist(vjets_results['Subleading_Jet_Eta'], bins=100, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[1, 1].hist(results['Subleading_Jet_Eta'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
axes[1, 1].set_xlabel(r'$\eta(j_{2})$', loc='right')
axes[1, 1].set_ylabel('Frecuency [A.U]')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

# Save individual plot for Subleading Jet Eta
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Subleading_Jet_Eta'], bins=100, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_results['Subleading_Jet_Eta'], bins=100, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(results['Subleading_Jet_Eta'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.xlabel(r'$\eta(j_{2})$', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/subleading_jet_eta_comparison.pdf')
plt.close()

# Plot 5: Dijet Mass comparison
axes[2, 0].hist(qcd_results['Dijet_Mass'], bins=100, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[2, 0].hist(vjets_results['Dijet_Mass'], bins=100, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[2, 0].hist(results['Dijet_Mass'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
# Add vertical line for Dijet Mass cut (large invariant mass)
axes[2, 0].axvline(x=800, color='black', linestyle=':', alpha=0.8, linewidth=2, label='mjj Cut (> 0.8 TeV)')
axes[2, 0].set_xlabel(r'$m_{jj}$ [GeV]', loc='right')
axes[2, 0].set_ylabel('Frecuency [A.U]')
axes[2, 0].set_yscale('log')
axes[2, 0].legend()
axes[2, 0].grid(True, alpha=0.3)

# Save individual plot for Dijet Mass
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Dijet_Mass'], bins=100, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_results['Dijet_Mass'], bins=100, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(results['Dijet_Mass'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.axvline(x=800, color='black', linestyle=':', alpha=0.8, linewidth=2, label='mjj Cut (> 0.8 TeV)')
plt.xlabel(r'$m_{jj}$ [GeV]', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/dijet_mass_comparison.pdf')
plt.close()

# Plot 6: Delta Eta comparison
axes[2, 1].hist(qcd_results['Delta_Eta_Jets'], bins=100, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[2, 1].hist(vjets_results['Delta_Eta_Jets'], bins=100, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[2, 1].hist(results['Delta_Eta_Jets'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
# Add vertical line for Delta Eta cut (large pseudorapidity separation)
axes[2, 1].axvline(x=3.8, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Δη Cut (> 3.8)')
axes[2, 1].set_xlabel(r'$\Delta\eta(j_{1},j_{2})$', loc='right')
axes[2, 1].set_ylabel('Frecuency [A.U]')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

# Save individual plot for Delta Eta
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Delta_Eta_Jets'], bins=100, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_results['Delta_Eta_Jets'], bins=100, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(results['Delta_Eta_Jets'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.axvline(x=3.8, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Δη Cut (> 3.8)')
plt.xlabel(r'$\Delta\eta(j_{1},j_{2})$', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/delta_eta_comparison.pdf')
plt.close()

# Plot 7: Delta Phi comparison
phi_bins = np.linspace(0, np.pi, 31)  # 30 bins from 0 to π
axes[3, 0].hist(qcd_results['Delta_Phi_Jets'], bins=phi_bins, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[3, 0].hist(vjets_results['Delta_Phi_Jets'], bins=phi_bins, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[3, 0].hist(results['Delta_Phi_Jets'], bins=phi_bins, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
# Add vertical line for Delta Phi cut (not back-to-back)
axes[3, 0].axvline(x=2, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Δφ Cut (< 2 rad)')
axes[3, 0].set_xlabel(r'$\Delta\phi(j_{1},j_{2})$ [rad]', loc='right')
axes[3, 0].set_ylabel('Frecuency [A.U]')
axes[3, 0].legend()
axes[3, 0].grid(True, alpha=0.3)

# Save individual plot for Delta Phi
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Delta_Phi_Jets'], bins=phi_bins, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_results['Delta_Phi_Jets'], bins=phi_bins, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(results['Delta_Phi_Jets'], bins=phi_bins, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.axvline(x=2, color='black', linestyle=':', alpha=0.8, linewidth=2, label='Δφ Cut (< 2 rad)')
plt.xlabel(r'$\Delta\phi(j_{1},j_{2})$ [rad]', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/delta_phi_comparison.pdf')
plt.close()

# Plot 8: Delta R comparison
axes[3, 1].hist(qcd_results['Delta_R_Jets'], bins=100, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[3, 1].hist(vjets_results['Delta_R_Jets'], bins=100, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[3, 1].hist(results['Delta_R_Jets'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
axes[3, 1].set_xlabel(r'$\Delta R(j_{1},j_{2})$', loc='right')
axes[3, 1].set_ylabel('Frecuency [A.U]')
axes[3, 1].legend()
axes[3, 1].grid(True, alpha=0.3)

# Save individual plot for Delta R
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['Delta_R_Jets'], bins=100, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_results['Delta_R_Jets'], bins=100, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(results['Delta_R_Jets'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.xlabel(r'$\Delta R(j_{1},j_{2})$', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/delta_r_comparison.pdf')
plt.close()

# Plot 9: MET_PT comparison
axes[4, 0].hist(qcd_results['MET_PT'], bins=100, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[4, 0].hist(vjets_results['MET_PT'], bins=100, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[4, 0].hist(results['MET_PT'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
# Add vertical line for MET cut
axes[4, 0].axvline(x=160, color='black', linestyle=':', alpha=0.8, linewidth=2, label='MET Cut (160 GeV)')
axes[4, 0].set_xlabel(r'$E_{T}^{miss}$ [GeV]', loc='right')
axes[4, 0].set_ylabel('Frecuency [A.U]')
axes[4, 0].set_yscale('log')
axes[4, 0].legend()
axes[4, 0].grid(True, alpha=0.3)

# Save individual plot for MET
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_results['MET_PT'], bins=100, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_results['MET_PT'], bins=100, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(results['MET_PT'], bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.axvline(x=160, color='black', linestyle=':', alpha=0.8, linewidth=2, label='MET Cut (160 GeV)')
plt.xlabel(r'$E_{T}^{miss}$ [GeV]', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.yscale('log')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/met_comparison.pdf')
plt.close()

# Plot 10: Product of Jet Etas comparison
vbf_eta_product = results['Leading_Jet_Eta'] * results['Subleading_Jet_Eta']
qcd_eta_product = qcd_results['Leading_Jet_Eta'] * qcd_results['Subleading_Jet_Eta']
vjets_eta_product = vjets_results['Leading_Jet_Eta'] * vjets_results['Subleading_Jet_Eta']

axes[4, 1].hist(qcd_eta_product, bins=100, alpha=0.8, label='QCD', color='red', 
                weights=qcd_results['event_weight'], histtype='step', linewidth=2)
axes[4, 1].hist(vjets_eta_product, bins=100, alpha=0.8, label='V+jets', color='green', 
                weights=vjets_results['event_weight'], histtype='step', linewidth=2)
axes[4, 1].hist(vbf_eta_product, bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
                weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
# Add vertical line for VBF topology requirement (opposite hemispheres)
axes[4, 1].axvline(x=0, color='black', linestyle=':', alpha=0.8, linewidth=2, label='VBF Cut (η₁ × η₂ < 0)')
axes[4, 1].set_xlabel(r'$\eta(j_{1}) \times \eta(j_{2})$', loc='right')
axes[4, 1].set_ylabel('Frecuency [A.U]')
axes[4, 1].legend()
axes[4, 1].grid(True, alpha=0.3)

# Save individual plot for Eta Product
fig_single = plt.figure(figsize=(5, 3.7))
plt.hist(qcd_eta_product, bins=100, alpha=0.8, label='QCD', color='red', 
         weights=qcd_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vjets_eta_product, bins=100, alpha=0.8, label='V+jets', color='green', 
         weights=vjets_results['event_weight'], histtype='step', linewidth=2)
plt.hist(vbf_eta_product, bins=100, alpha=0.8, label='p p → j j h, h → inv', color='blue', 
         weights=results['event_weight'], histtype='step', linewidth=2, linestyle='--')
plt.axvline(x=0, color='black', linestyle=':', alpha=0.8, linewidth=2, label='VBF Cut (η₁ × η₂ < 0)')
plt.xlabel(r'$\eta(j_{1}) \times \eta(j_{2})$', loc='right')
plt.ylabel('Frecuency [A.U]')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('Images/eta_product_comparison.pdf')
plt.close()

# Save the combined plot
plt.tight_layout()
plt.savefig('Images/vbf_qcd_vjets_comparison.pdf')
plt.show()

# Helper functions for weighted statistics
def weighted_mean(values, weights):
    return np.average(values, weights=weights)

def weighted_std(values, weights):
    wmean = weighted_mean(values, weights)
    return np.sqrt(np.average((values - wmean)**2, weights=weights))

print("Individual plots saved as separate PDFs in HiggsPortals/Images/ directory:")
print("- leading_jet_pt_comparison.pdf")
print("- subleading_jet_pt_comparison.pdf")
print("- leading_jet_eta_comparison.pdf")
print("- subleading_jet_eta_comparison.pdf")
print("- dijet_mass_comparison.pdf")
print("- delta_eta_comparison.pdf")
print("- delta_phi_comparison.pdf")
print("- delta_r_comparison.pdf")
print("- met_comparison.pdf")
print("- eta_product_comparison.pdf")
print("- vbf_qcd_vjets_comparison.pdf (combined plot)")

In [ ]:
# Define VBF cuts query
vbf_cuts = "Leading_Jet_PT > 80 and Subleading_Jet_PT > 50 and Dijet_Mass > 800 and Delta_Eta_Jets > 3.8 and Delta_Phi_Jets < 2 and MET_PT > 160 and (Leading_Jet_Eta * Subleading_Jet_Eta) < 0"

# Apply all VBF cuts to create filtered dataset
vbf_filtered = results.query(vbf_cuts)

print(f"Original VBF events: {len(results)}")
print(f"After all cuts: {len(vbf_filtered)}")
print(f"Selection efficiency: {len(vbf_filtered)/len(results)*100:.2f}%")

# Also create filtered datasets for comparison
qcd_filtered = qcd_results.query(vbf_cuts)

vjets_filtered = vjets_results.query(vbf_cuts)

print(f"\nComparison after all cuts:")
print(f"VBF events: {len(vbf_filtered)}")
print(f"QCD events: {len(qcd_filtered)}")
print(f"V+jets events: {len(vjets_filtered)}")

In [ ]:
# Create MET comparison plot after all cuts
plt.figure(figsize=(5, 3.5))

# Calculate efficiencies (integral after cuts / integral before cuts)
vbf_efficiency = vbf_filtered['event_weight'].sum() / results['event_weight'].sum()
qcd_efficiency = qcd_filtered['event_weight'].sum() / qcd_results['event_weight'].sum()
vjets_efficiency = vjets_filtered['event_weight'].sum() / vjets_results['event_weight'].sum()

# Plot histograms with range 160-960 GeV using step format
plt.hist(qcd_filtered['Dijet_Mass'], bins=30, alpha=0.8, 
         label=f'QCD (eff: {qcd_efficiency:.2e})', color='red', 
         weights=qcd_filtered['event_weight'], range=(800, 7500), histtype='step', linewidth=2)
plt.hist(vjets_filtered['Dijet_Mass'], bins=30, alpha=0.8, 
         label=f'V+jets (eff: {vjets_efficiency:.2e})', color='green', 
         weights=vjets_filtered['event_weight'], range=(800, 7500), histtype='step', linewidth=2)
plt.hist(vbf_filtered['Dijet_Mass'], bins=30, alpha=0.8, 
         label=f'p p → j j h, h → inv (eff: {vbf_efficiency:.2e})', color='blue', 
         weights=vbf_filtered['event_weight'], range=(800, 7500), histtype='step', linewidth=2, linestyle='--')

# Add vertical line for MET cut (though all events should pass this by definition)
plt.axvline(x=160, color='black', linestyle=':', alpha=0.8, linewidth=2, label='MET Cut (160 GeV)')

plt.xlabel(r'$m_{jj}$ [GeV]', loc='right')
plt.ylabel('Events Passing Cuts / Total MC Events')

plt.title('Dijet Mass Comparison After VBF Cuts')
plt.xlim(800, 7500)  # Set x-axis limits
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Save the plot
plt.savefig('Images/met_comparison_after_vbf_cuts.pdf')

# Print efficiency summary
print(f"\nVBF Cut Efficiencies:")
print(f"VBF: {vbf_efficiency:.2e}")
print(f"QCD: {qcd_efficiency:.2e}")
print(f"V+jets: {vjets_efficiency:.2e}")